The notebook shows how fundamental concepts of pagerank can be translated into code. I use 'Mining of Massive Datasets' book by Jure Leskovec, Anand Rajaraman and Jeff Ullman as a reference.

Link to the book: http://www.mmds.org/

I assume that reader has a basic understanding of Pagerank. If not, you may want to look at the related chapter of 'Mining of Massive Datasets' book.

## Coverage

0. Introduction
  - 0.1. Web/Graph Representation
1. Simplified PageRank
2. Dead-ends and Spider Traps
3. PageRank with Random Teleportation (Taxation)
4. Spam-Farms
5. TrustRank
6. Apply PageRank on a Real Dataset
7. One More Optimization
- References

## 0. Introduction

### 0.1. Web/Graph Representation

Given a directed graph, we'll represent it as dictionary of lists. Each node of the graph will be a key of the dictionary (`G`) and outgoing edges of a particular node `u` will be a list pointed by `G[u]`. Note that, even though a node may not have any outgoing links we still assign a value (empty list) to that key in the dictionary.

Figure - Toy Graph:
![1.png](1.png)

In [1]:
def toy_graph():
    G = dict()
    G[0] = [1, 2, 3]
    G[1] = [0, 3]
    G[2] = [0]
    G[3] = [1, 2]
    return G

## 1. Simplified Pagerank 

Pagerank problem is simply finding stationary distribution of states in a markov model.

In [2]:
def pagerank(G, iteration_count=100):
    
    N = len(G.keys())
    next_rank_lst = [1/N for _ in range(N)]
    current_rank_lst = next_rank_lst[:]
    
    for i in range(iteration_count):
        current_rank_lst, next_rank_lst = next_rank_lst, current_rank_lst
        for j in range(N):
            next_rank_lst[j] = 0
        for node in G:
            if G[node]: # To avoid division by zero problem in the next line
                contribution = current_rank_lst[node] / len(G[node])
                for edge in G[node]:
                    next_rank_lst[edge] += contribution
    
    return next_rank_lst

In [3]:
G = toy_graph()
rank_lst = pagerank(G)
print(rank_lst)
print(sum(rank_lst)) # Total rank is equal to 1

[0.3333333333333333, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222]
1.0


## 2. Dead-ends and Spider Traps

The main problems of simplified pagerank are dead-end nodes and spider traps structures.

### 2.1 Dead-ends

Dead-end is node with no outgoing edges. Since page importance has nowhere to be transferred, such nodes cause rank leakage.

![2.png](2.png)

Importance of node 2 has nowhere to go.

In [4]:
G = toy_graph()
G[2].clear() # By removing edges of node 2, we create a dead-end

In [5]:
rank_lst = pagerank(G)
print(rank_lst)
print(sum(rank_lst)) # Total rank is less than 1 due to rank leakage

[3.4282767441682065e-15, 4.996463459841385e-15, 4.996463459841385e-15, 4.996463459841385e-15]
1.841766712369236e-14


### 2.2 Spider Traps

In [6]:
G = toy_graph()
G[2] = [2] # Node 2 absorbes all importance

In [7]:
rank_lst = pagerank(G)
print(rank_lst)
print(sum(rank_lst))

[3.4282767441682065e-15, 4.996463459841385e-15, 0.9999999999999868, 4.996463459841385e-15]
1.0000000000000002


In [8]:
G = toy_graph()
G[2] = [4]
G[4] = [2] # Node 2 and 4 absorb all importance

In [9]:
rank_lst = pagerank(G)
print(rank_lst)
print(sum(rank_lst))

[2.742621395334566e-15, 3.997170767873109e-15, 0.4499999999999953, 3.997170767873109e-15, 0.5499999999999937]
0.9999999999999998


## 3. Pagerank with Random Teleportation

In [10]:
def pagerank(G, beta=0.85, iteration_count=100):
    
    N = len(G.keys())
    next_rank_lst = [1/N for _ in range(N)]
    current_rank_lst = next_rank_lst[:]
    
    for i in range(iteration_count):
        current_rank_lst, next_rank_lst = next_rank_lst, current_rank_lst
        for j in range(N):
            next_rank_lst[j] = (1 - beta) / N
        for node in G:
            if G[node]:
                contribution = beta * (current_rank_lst[node] / len(G[node]))
                for edge in G[node]:
                    next_rank_lst[edge] += contribution
        
        leakage_contribution = (1 - sum(next_rank_lst)) / N
        for j in range(N):
            next_rank_lst[j] += leakage_contribution
    
    return next_rank_lst

In [11]:
G = toy_graph()
G[2].clear()

In [12]:
rank_lst = pagerank(G, beta=0.8)
print(rank_lst)
print(sum(rank_lst))

[0.20833333333333334, 0.2638888888888889, 0.2638888888888889, 0.2638888888888889]
1.0


In [13]:
G = toy_graph()
G[2] = [2]

In [14]:
rank_lst = pagerank(G, beta=0.8)
print(rank_lst)
print(sum(rank_lst))

[0.10135135135135134, 0.12837837837837837, 0.6418918918918919, 0.12837837837837837]
1.0


## 4. Spam Farms

In [15]:
G = toy_graph()
G[2].clear()
for i in range(len(G.keys()), 100):
    G[i] = [2]
    G[2].append(i)

In [16]:
rank_lst = pagerank(G, beta=0.8)
print(rank_lst[:4])

[0.004054054054054079, 0.005135135135135165, 0.4409309308444677, 0.005135135135135165]


## 5. TrustRank

In [17]:
def pagerank(G, beta=0.85, iteration_count=100, teleport_lst=None):
    
    if not teleport_lst:
        teleport_lst = G.keys()
    
    N = len(G.keys())
    next_rank_lst = [1/N for _ in range(N)]
    current_rank_lst = next_rank_lst[:]
    
    teleport_lst_count = len(teleport_lst)
    
    for i in range(iteration_count):
        current_rank_lst, next_rank_lst = next_rank_lst, current_rank_lst
        for j in range(N):
            next_rank_lst[j] = 0
        for node in teleport_lst:
            next_rank_lst[node] = (1 - beta) / teleport_lst_count
        for node in G:
            if G[node]:
                contribution = beta * (current_rank_lst[node] / len(G[node]))
                for edge in G[node]:
                    next_rank_lst[edge] += contribution
        
        leakage_contribution = (1 - sum(next_rank_lst)) / N
        for j in range(N):
            next_rank_lst[j] += leakage_contribution
        
    return next_rank_lst

In [18]:
G = toy_graph()
G[2].clear()
for i in range(len(G.keys()), 100):
    G[i] = [2]
    G[2].append(i)

trust_lst = [1]
rank_lst = pagerank(G, beta=0.8, teleport_lst=trust_lst)
print(rank_lst[:4])

[0.11583011583011586, 0.28957528957528955, 0.2488202487173264, 0.14671814671814676]


## 6. Apply PageRank on a Real Dataset

In [19]:
# Download dataset

import gzip
import shutil
import urllib
import os

url = 'https://snap.stanford.edu/data/web-NotreDame.txt.gz'

filename = 'data.txt'
gzip_file = '%s.gz' % filename

# Download the dataset
is_download_required = not os.path.isfile(filename) 

if is_download_required:
    urllib.request.urlretrieve(url, gzip_file)
    
    with gzip.open(gzip_file, 'rb') as f_in:
        with open(filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [20]:
# Parse input file
G = dict()

with open(filename, 'r') as file:
    for line in file:
        if line[0] == '#':
            continue
        
        u, v = map(int, line.split())
        
        if u not in G:
            G[u] = list()
        
        G[u].append(v)

N = max(G.keys()) + 1
for i in range(N):
    if i not in G:
        G[i] = list()

In [21]:
rank_lst = pagerank(G)

In [25]:
rank_lst[:10]

[0.0054665478909549335,
 0.00047967433074845605,
 0.0002750863819726011,
 0.0003679031647468208,
 0.0003595245611761849,
 0.000304996125513151,
 0.0002926141797057847,
 0.00030023960928669726,
 0.0002836810879649729,
 0.0002861092013107728]

## 7. One More Optimization

In [23]:
def pagerank(G, beta=0.85, iteration_count=100, teleport_lst=None, eps=1e-8):
    
    if not teleport_lst:
        teleport_lst = G.keys()
    
    N = len(G.keys())
    next_rank_lst = [1/N for _ in range(N)]
    current_rank_lst = next_rank_lst[:]
    
    teleport_lst_count = len(teleport_lst)
    
    for i in range(iteration_count):
        current_rank_lst, next_rank_lst = next_rank_lst, current_rank_lst
        for j in range(N):
            next_rank_lst[j] = 0
        for node in teleport_lst:
            next_rank_lst[node] = (1 - beta) / teleport_lst_count
        for node in G:
            if G[node]:
                contribution = beta * (current_rank_lst[node] / len(G[node]))
                for edge in G[node]:
                    next_rank_lst[edge] += contribution
        
        leakage_contribution = (1 - sum(next_rank_lst)) / N
        for j in range(N):
            next_rank_lst[j] += leakage_contribution
        
        total_diff = 0
        for c, n in zip(current_rank_lst, next_rank_lst):
            total_diff += abs(c - n)
        
        if total_diff < eps:
            return next_rank_lst
    
    return next_rank_lst

In [24]:
rank_lst = pagerank(G)

## References

 - Rajaraman, Anand, and Jeffrey David Ullman. Mining of massive datasets. Cambridge University Press, 2011.
